In [3]:
# tf-idf preprocess
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

print('reading...')
df = pd.read_csv('./train_merged.csv')
print('extracting...')
creative_id = df.groupby('user_id')['creative_id'].apply(list)
advertiser_id = df.groupby('user_id')['advertiser_id'].apply(list)
ad_id = df.groupby('user_id')['ad_id'].apply(list)
del df
print('building first tfidf...')
v_creat = TfidfVectorizer(lowercase=False,tokenizer=lambda x:x)
X_creat = v_creat.fit_transform(creative_id)
del creative_id
print('building second tfidf...')
v_adver = TfidfVectorizer(lowercase=False,tokenizer=lambda x:x)
X_adver = v_adver.fit_transform(advertiser_id)
del advertiser_id
print('building third tfidf...')
v_adid = TfidfVectorizer(lowercase=False,tokenizer=lambda x:x)
X_adid = v_adid.fit_transform(ad_id)
del ad_id
print('saving train attrubutes...')
train_attr = sparse.hstack((X_creat, X_adver, X_adid), dtype=float)
del X_creat, X_adver, X_adid
sparse.save_npz('./train_attr.npz', train_attr)
del train_attr

print('preprocessing predict data...')
print('reading...')
df = pd.read_csv('./predict_merged.csv')
print('extracting...')
creative_id = df.groupby('user_id')['creative_id'].apply(list)
advertiser_id = df.groupby('user_id')['advertiser_id'].apply(list)
ad_id = df.groupby('user_id')['ad_id'].apply(list)
del df
print('building first tfidf...')
X_creat = v_creat.transform(creative_id)
del creative_id
print('building second tfidf...')
X_adver = v_adver.transform(advertiser_id)
del advertiser_id
print('building third tfidf...')
X_adid = v_adid.transform(ad_id)
del ad_id
print('saving predict attrubutes...')
train_attr = sparse.hstack((X_creat, X_adver, X_adid), dtype=float)
del X_creat, X_adver, X_adid
sparse.save_npz('./predict_attr.npz', train_attr)
del train_attr

building third tfidf...
saving train attrubutes...
preprocessing predict data...
reading...
extracting...
building first tfidf...
building second tfidf...
building third tfidf...
saving train attrubutes...


In [2]:
# predict gender
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse
from sklearn.model_selection import KFold
import numpy as np

print('loading data...')
X_train = sparse.load_npz('./train_attr.npz').tocsr()
X_predict = sparse.load_npz('./predict_attr.npz').tocsr()
y_train = pd.read_csv('./user_info.csv')['gender']
print('data is prepared.')

predictions = np.zeros(X_predict.shape[0], dtype=float)
folds = KFold(n_splits=5, shuffle=False)

print('start training...')
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    display(trn_idx,val_idx)
    trn_data_X_train=X_train[trn_idx,:]
    val_data_X_train=X_train[val_idx,:]
    display(trn_data_X_train,val_data_X_train)
    trn_data = lgb.Dataset(trn_data_X_train, y_train.iloc[trn_idx])
    val_data = lgb.Dataset(val_data_X_train, y_train.iloc[val_idx])
    
    lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':8,
             }
    model = lgb.train(lgb_params, 
                    trn_data,
                    num_boost_round=200, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval = 10, 
                    early_stopping_rounds = 100)
    predictions += model.predict(X_predict, num_iteration=model.best_iteration) / folds.n_splits
    display(predictions)
    display(np.where(predictions>0.5))

display(predictions)

loading data...
data is prepared.
start training...
fold n°1


array([180000, 180001, 180002, ..., 899997, 899998, 899999])

array([     0,      1,      2, ..., 179997, 179998, 179999])

<720000x4797415 sparse matrix of type '<class 'numpy.float64'>'
	with 61014420 stored elements in Compressed Sparse Row format>

<180000x4797415 sparse matrix of type '<class 'numpy.float64'>'
	with 15292962 stored elements in Compressed Sparse Row format>

...

<180000x4797415 sparse matrix of type '<class 'numpy.float64'>'
	with 15260737 stored elements in Compressed Sparse Row format>

Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.906804	valid_1's auc: 0.906557
[20]	training's auc: 0.92273	valid_1's auc: 0.921658
[30]	training's auc: 0.933957	valid_1's auc: 0.932807
[40]	training's auc: 0.941481	valid_1's auc: 0.940373
[50]	training's auc: 0.946573	valid_1's auc: 0.945301
[60]	training's auc: 0.950208	valid_1's auc: 0.948763
[70]	training's auc: 0.952912	valid_1's auc: 0.951279
[80]	training's auc: 0.955106	valid_1's auc: 0.95333
[90]	training's auc: 0.956925	valid_1's auc: 0.954951
[100]	training's auc: 0.95843	valid_1's auc: 0.956325
[110]	training's auc: 0.959697	valid_1's auc: 0.957433
[120]	training's auc: 0.960791	valid_1's auc: 0.958387
[130]	training's auc: 0.961792	valid_1's auc: 0.959199
[140]	training's auc: 0.962662	valid_1's auc: 0.959953
[150]	training's auc: 0.963461	valid_1's auc: 0.96062
[160]	training's auc: 0.964222	valid_1's auc: 0.961202
[170]	training's auc: 0.964912	valid_1's auc: 0.961733
[180]	training's

array([0.01262285, 0.99449394, 0.32285698, ..., 0.00637901, 0.00173432,
       0.00263036])

(array([     1,      9,     13, ..., 999992, 999993, 999994]),)

array([0.01262285, 0.99449394, 0.32285698, ..., 0.00637901, 0.00173432,
       0.00263036])

In [3]:
submission=pd.DataFrame([i for i in range(3000001,4000001)],columns=["user_id"])
submission["predicted_age"]=[0 for _ in range(1000000)]
submission["predicted_gender"]=(predictions>0.5)*1
submission["predicted_gender"]+=1
display(submission)
submission.describe()
submission.to_csv("submission.csv",index=False)

,user_id,predicted_age,predicted_gender
0,3000001,0,1
1,3000002,0,2
2,3000003,0,1
3,3000004,0,1
4,3000005,0,1
...,...,...,...
999995,3999996,0,1
999996,3999997,0,1
999997,3999998,0,1
999998,3999999,0,1


In [5]:
# predict age
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse
from sklearn.model_selection import KFold
import numpy as np

print('loading data...')
X_train = sparse.load_npz('./train_attr.npz').tocsr()
X_predict = sparse.load_npz('./predict_attr.npz').tocsr()
y_train = pd.read_csv('./user_info.csv')['age']
print('data is prepared.')

predictions = np.zeros(X_predict.shape[0], dtype=float)
folds = KFold(n_splits=5, shuffle=False)

print('start training...')
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    display(trn_idx,val_idx)
    trn_data_X_train=X_train[trn_idx,:]
    val_data_X_train=X_train[val_idx,:]
    display(trn_data_X_train,val_data_X_train)
    trn_data = lgb.Dataset(trn_data_X_train, y_train.iloc[trn_idx])
    val_data = lgb.Dataset(val_data_X_train, y_train.iloc[val_idx])
    
    lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'multiclass',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':8,
             }
    model = lgb.train(lgb_params, 
                    trn_data,
                    num_boost_round=200, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval = 10, 
                    early_stopping_rounds = 100)
    predictions += model.predict(X_predict, num_iteration=model.best_iteration) / folds.n_splits
    display(predictions)

display(predictions)

<1x4797415 sparse matrix of type '<class 'numpy.float64'>'
	with 36 stored elements in Compressed Sparse Row format>

In [ ]:
import pandas as pd
import numpy as np

submission = pd.read_csv('./submission.csv')
predictions = np.around(predictions)
predictions[predictions > 10] = 10
predictions[predictions < 1] = 1
submission['age'] = predictions
submission.to_csv('./submission.csv')